In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.time import Time
import astropy.units as u
from astropy.units import cds
from lightkurve import LightCurve

from gadfly import PowerSpectrum, SolarOscillatorKernel

In [ ]:
hdu = fits.open('../data/VIRGO_1min_0083-7404.fits.gz')
raw_fluxes = hdu[0].data
header = hdu[0].header

header

In [ ]:
soho_mission_day = Time("1995-12-1 00:00")

times = (
    soho_mission_day.jd + 
    header['TIME'] + 
    np.arange(header['NAXIS1']) / 1440
)
times_astropy = Time(times, format='jd')

fluxes = raw_fluxes.copy()
interp_fluxes = np.interp(
    times[raw_fluxes == -99], times[raw_fluxes != -99], fluxes[raw_fluxes != -99]
)
d = (times[1] - times[0]) * u.day

fluxes[raw_fluxes == -99] = interp_fluxes

fluxes = 1e6 * (fluxes / np.median(fluxes) - 1) * u.cds.ppm

light_curve = LightCurve(time=times_astropy, flux=fluxes)

In [ ]:
ps = PowerSpectrum.from_light_curve(light_curve, detrend=False)

In [ ]:
binning_500 = ps.cutout(frequency_max=4500*u.uHz).bin(100)

In [ ]:
cutout = ps.cutout(1800*u.uHz, 4500*u.uHz).bin(5_000, log=False)

In [ ]:
kernel_freq_grid = np.sort(np.concatenate([
    np.logspace(np.log10(0.05), np.log10(5000), 500),
    np.linspace(1800, 4500, 10_000)
])) * u.uHz

In [ ]:
fig, ax = binning_500.plot(
    freq=kernel_freq_grid,
    obs_kwargs=dict(marker='.', color='k'), 
    figsize=(8, 6), 
    label_obs='SOHO VIRGO/PMO6',
    create_new_figure=True,
)
cutout.plot(
    ax=ax,
    freq=kernel_freq_grid,
    kernel=SolarOscillatorKernel(),
    p_mode_inset=True,
    inset_bounds=[0.4, 0.6, 0.55, 0.35],
    obs_kwargs=dict(marker=None), 
    label_obs='',
    label_kernel='gadfly',
    kernel_kwargs=dict(color='r', alpha=1, lw=1.5, zorder=-10, rasterized=True),
    inset_kwargs=dict(marker='.', mfc='none', color='k', zorder=10, rasterized=True),
)
ax.set(
    xlim=[0.04, 1e4],
    ylim=[5e-3, 2e6],
    yticks=10.0 ** np.arange(-2, 7)
)
ax.legend(loc='lower left')

for sp in ['right', 'top']:
    ax.spines[sp].set_visible(False)
    
fig.savefig('plots/soho_virgo.pdf', bbox_inches='tight', dpi=250)